In [61]:
module Ev3dev

import Base: read, parse, write

parse(::Type{ASCIIString}, s::ASCIIString) = s
parse(::Type{Vector{ASCIIString}}, s::ASCIIString) = split(s)

immutable Brick
    root_path::AbstractString
    
    Brick(root_path::AbstractString="/") = new(root_path)
end

immutable Attribute{T, Read, Write}
    name::Symbol
    stream::IOStream
end

function call{T, Read, Write}(::Type{Attribute{T, Read, Write}}, brick::Brick, relative_path::AbstractString, name)
    path = joinpath(brick.root_path, relative_path, string(name))
    isfile(path) || error("file not found: $path")
    Read && (isreadable(path) || error("read access for attribute $(path) was requested, but file is not readable"))
    Write && (iswritable(path) || error("write access for attribute $(path) was requested, but file is not writable"))
    stream = open(path, Read, Write, false, Write, false)
    Attribute{T, Read, Write}(name, stream)
end

typealias ReadOnly{T} Attribute{T, true, false}
typealias WriteOnly{T} Attribute{T, false, true}
typealias ReadWrite{T} Attribute{T, true, true}
    
call{T, X}(attr::Attribute{T, true, X}) = parse(T, readchomp(seekstart(attr.stream)))
function call{T, X}(attr::Attribute{T, X, true}, value::T)
    write(seekstart(attr.stream), string(value))
    flush(attr.stream)
end

abstract Device
abstract AttributeSet

type MotorAttributes <: AttributeSet
    address::ReadOnly{ASCIIString}
    commands::ReadOnly{Vector{ASCIIString}}
    command::WriteOnly{ASCIIString}
    speed_sp::ReadWrite{Int}
    
    MotorAttributes() = new()
    
end
type Motor <: Device
    attr::MotorAttributes
    Motor(brick, path) = new(MotorAttributes(brick, path))
end

type SensorAttributes <: AttributeSet
    address::ReadOnly{ASCIIString}
    commands::ReadOnly{Vector{ASCIIString}}
    command::WriteOnly{ASCIIString}
    value0::ReadOnly{Int}
    decimals::ReadOnly{Int}
    
    SensorAttributes() = new()
end
type Sensor <: Device
    attr::SensorAttributes
    Sensor(brick, path) = new(SensorAttributes(brick, path))
end
    

function call{T <: AttributeSet}(::Type{T}, brick::Brick, path::AbstractString)
    attr = T()
    for name in fieldnames(T)
        field_type = fieldtype(T, name)
        setfield!(attr, name, field_type(brick, path, name))
    end
    attr
end 

end

import Ev3dev

In [63]:
brick = Ev3dev.Brick("/home/pi/ev3")
motor = Ev3dev.Motor(brick, "sys/class/tacho-motor/motor0")
sensor = Ev3dev.Sensor(brick, "sys/class/lego-sensor/sensor0")

Ev3dev.Sensor(Ev3dev.SensorAttributes(Ev3dev.Attribute{ASCIIString,true,false}(:address,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/address>)),Ev3dev.Attribute{Array{ASCIIString,1},true,false}(:commands,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/commands>)),Ev3dev.Attribute{ASCIIString,false,true}(:command,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/command>)),Ev3dev.Attribute{Int32,true,false}(:value0,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value0>)),Ev3dev.Attribute{Int32,true,false}(:decimals,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/decimals>))))

In [66]:
sensor.attr.value0()

569

In [60]:
motor.attr.address()

"outA"

In [55]:
motor.attr.speed_sp(500)

IOStream(<file /home/pi/ev3/sys/class/tacho-motor/motor0/speed_sp>)

In [56]:
motor.attr.command("run-forever")

IOStream(<file /home/pi/ev3/sys/class/tacho-motor/motor0/command>)

In [57]:
motor.attr.command("stop")

IOStream(<file /home/pi/ev3/sys/class/tacho-motor/motor0/command>)